In [ ]:
import os
import numpy as np
caffe_root = '/home/flathead/bitbucket/mycaffe/'
os.chdir('../../')
import sys
sys.path.insert(0, 'python')
import caffe

In [ ]:
from pylab import *
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from numpy.random import normal

In [ ]:
caffe.set_mode_gpu()
caffe.set_device(0)

In [ ]:
#solver = None  # ignore this workaround for lmdb data (can't instantiate two solvers on the same data)
#solver = caffe.SGDSolver('examples/mlp/mlp_solver.prototxt')
solver = caffe.AdamSolver('examples/mlp/mlp_solver.prototxt')

In [ ]:
net1 = solver.net
test_net = solver.test_nets[0]

In [ ]:
# during training, each output is (batch size, feature dim, spatial dim)
[(k, v.data.shape) for k, v in net1.blobs.items()]

In [ ]:
# just print the weight sizes (we'll omit the biases)
[(k, v[0].data.shape) for k, v in net1.params.items()]

In [ ]:
test_net.forward()

In [ ]:
feat1 = net1.params['ip1'][0].data
feat2 = net1.params['ip2'][0].data
feat3 = net1.params['ip3'][0].data
feat4 = net1.params['ip4'][0].data

In [ ]:
l1 = 500
l2 = 500
l3 = 2000
l4 = 10

In [ ]:
%%time
niter = 50000
test_interval = 5000
train_loss = zeros(niter)
test_acc = zeros(int(np.ceil(niter / test_interval)))
output = zeros((niter, 8, 10))

for it in range(niter):
    solver.step(1)
    #net1.forward()
    
    train_loss[it] = net1.blobs['loss'].data
    
    net1.forward(start='ip1')
    output[it] = test_net.blobs['ip4'].data[:8]

    if it % test_interval == 0:
        print 'iteration', it, 'testing...'
        correct = 0
        for test_it in range(100):
            test_net.forward()
            correct += sum(test_net.blobs['ip4'].data.argmax(1) == test_net.blobs['label'].data)
        test_acc[it // test_interval] = correct / 1e4

In [ ]:
net1.save('examples/mlp/baseline.caffemodel')

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(311)
plt.hist(feat1[0].flat, bins= 500, range=(-1, 1))
plt.title("ip1_weight")

In [ ]:
#Neurons of Layer 1
mean_feat1 = [0] * l1
std_feat1 = [0] * l1

for j in range(l1):
    mean_feat1[j] = feat1[j].mean()
    std_feat1[j] = feat1[j].std()
    
#Neurons of Layer 2
mean_feat2 = [0] * l2
std_feat2 = [0] * l2

for j in range(l2):
    mean_feat2[j] = feat2[j].mean()
    std_feat2[j] = feat2[j].std()
    
#Neurons of Layer 3
mean_feat3 = [0] * l3
std_feat3 = [0] * l3

for j in range(l3):
    mean_feat3[j] = feat3[j].mean()
    std_feat3[j] = feat3[j].std()
    
#Neurons of Layer 4
mean_feat4 = [0] * l4
std_feat4 = [0] * l4

for j in range(l4):
    mean_feat4[j] = feat4[j].mean()
    std_feat4[j] = feat4[j].std()

In [ ]:
#Neurons of Layer 1
posm_feat1 = [0] * l1
negm_feat1 = [0] * l1
poss_feat1 = [0] * l1
negs_feat1 = [0] * l1

for j in range(l1):
    posm_feat1[j] = feat1[j][feat1[j] > 0].mean()
    negm_feat1[j] = feat1[j][feat1[j] < 0].mean()
    poss_feat1[j] = feat1[j][feat1[j] > 0].std()
    negs_feat1[j] = feat1[j][feat1[j] < 0].std()
    
#Neurons of Layer 2
posm_feat2 = [0] * l2
negm_feat2 = [0] * l2
poss_feat2 = [0] * l2
negs_feat2 = [0] * l2

for j in range(l2):
    posm_feat2[j] = feat2[j][feat2[j] > 0].mean()
    negm_feat2[j] = feat2[j][feat2[j] < 0].mean()
    poss_feat2[j] = feat2[j][feat2[j] > 0].std()
    negs_feat2[j] = feat2[j][feat2[j] < 0].std()
    
#Neurons of Layer 3
posm_feat3 = [0] * l3
negm_feat3 = [0] * l3
poss_feat3 = [0] * l3
negs_feat3 = [0] * l3

for j in range(l3):
    posm_feat3[j] = feat3[j][feat3[j] > 0].mean()
    negm_feat3[j] = feat3[j][feat3[j] < 0].mean()
    poss_feat3[j] = feat3[j][feat3[j] > 0].std()
    negs_feat3[j] = feat3[j][feat3[j] < 0].std()

#Neurons of Layer 4
posm_feat4 = [0] * l4
negm_feat4 = [0] * l4
poss_feat4 = [0] * l4
negs_feat4 = [0] * l4

for j in range(l4):
    posm_feat4[j] = feat4[j][feat4[j] > 0].mean()
    negm_feat4[j] = feat4[j][feat4[j] < 0].mean()
    poss_feat4[j] = feat4[j][feat4[j] > 0].std()
    negs_feat4[j] = feat4[j][feat4[j] < 0].std()

In [ ]:
# rate * std range zero pruning
rate = 0.3

a1 = [0] * l1
b1 = [0] * l2
c1 = [0] * l3
d1 = [0] * l4
a2 = [0] * l1
b2 = [0] * l2
c2 = [0] * l3
d2 = [0] * l4

for x in range(l1):
    a1[x] = (posm_feat1[x] - 0) * rate
    a2[x] = (0 - negm_feat1[x]) * rate
for y in range(l2):
    b1[y] = (posm_feat2[y] - 0) * rate
    b2[y] = (0 - negm_feat2[y]) * rate
for z in range(l3):
    c1[z] = (posm_feat3[z] - 0) * rate
    c2[z] = (0 - negm_feat3[z]) * rate
for u in range(l4):
    d1[u] = (posm_feat4[u] - 0) * rate
    d2[u] = (0 - negm_feat4[u]) * rate

for x in range(l1):
    for y in range(784):
        #if ((feat1[x,y] >= (mean_feat1[x] - a2[x] * std_feat1[x])) & (feat1[x,y] <= (mean_feat1[x] + a1[x] * std_feat1[x]))):
        if ((feat1[x,y] >= -a2[x]) & (feat1[x,y] <= a1[x])):
        #if ((feat1[x,y] >= -std_feat1[x] * rate) & (feat1[x,y] <= rate * std_feat1[x])):
            feat1[x,y] = 0
            
for x in range(l2):
    for y in range(l1):
        #if (feat2[x,y] >= (mean_feat2[x] - b2[x] * std_feat2[x])) & (feat2[x,y] <= (mean_feat2[x] + b1[x] * std_feat2[x])):
        if (feat2[x,y] >= -b2[x]) & (feat2[x,y] <= b1[x]):
        #if ((feat2[x,y] >= -std_feat2[x] * rate) & (feat2[x,y] <= rate * std_feat2[x])):
            feat2[x,y] = 0
            
for x in range(l3):
    for y in range(l2):
        #if (feat3[x,y] >= (mean_feat3[x] - c2[x] * std_feat3[x])) & (feat3[x,y] <= (mean_feat3[x] + c1[x] * std_feat3[x])):
        #if (feat3[x,y] >= -c2[x]) & (feat3[x,y] <= c1[x]):
        if ((feat3[x,y] >= -std_feat3[x] * rate) & (feat3[x,y] <= std_feat3[x] * rate)):
            feat3[x,y] = 0
            
for x in range(l4):
    for y in range(l3):
        #if (feat4[x,y] >= (mean_feat4[x] - d2[x] * std_feat4[x])) & (feat4[x,y] <= (mean_feat4[x] + d1[x] * std_feat4[x])):
        #if (feat4[x,y] >= -d2[x]) & (feat4[x,y] <= d1[x]):
        if ((feat4[x,y] >= -std_feat4[x] * rate) & (feat4[x,y] <= std_feat4[x] * rate)):
            feat4[x,y] = 0

In [ ]:
for x in range(l1):
    print (feat1[x] == 0).sum()
for y in range(l2):
    print (feat2[y] == 0).sum()
for z in range(l3):
    print (feat3[z] == 0).sum()
for u in range(l4):
    print (feat4[u] == 0).sum()

In [ ]:
net1.save('examples/mlp/zeropr/pruned.caffemodel')

In [ ]:
#Neurons of Layer 1
posm_feat1 = [0] * l1
negm_feat1 = [0] * l1
poss_feat1 = [0] * l1
negs_feat1 = [0] * l1

for j in range(l1):
    posm_feat1[j] = feat1[j][feat1[j] > 0].mean()
    negm_feat1[j] = feat1[j][feat1[j] < 0].mean()
    poss_feat1[j] = feat1[j][feat1[j] > 0].std()
    negs_feat1[j] = feat1[j][feat1[j] < 0].std()
    
#Neurons of Layer 2
posm_feat2 = [0] * l2
negm_feat2 = [0] * l2
poss_feat2 = [0] * l2
negs_feat2 = [0] * l2

for j in range(l2):
    posm_feat2[j] = feat2[j][feat2[j] > 0].mean()
    negm_feat2[j] = feat2[j][feat2[j] < 0].mean()
    poss_feat2[j] = feat2[j][feat2[j] > 0].std()
    negs_feat2[j] = feat2[j][feat2[j] < 0].std()
    
#Neurons of Layer 3
posm_feat3 = [0] * l3
negm_feat3 = [0] * l3
poss_feat3 = [0] * l3
negs_feat3 = [0] * l3

for j in range(l3):
    posm_feat3[j] = feat3[j][feat3[j] > 0].mean()
    negm_feat3[j] = feat3[j][feat3[j] < 0].mean()
    poss_feat3[j] = feat3[j][feat3[j] > 0].std()
    negs_feat3[j] = feat3[j][feat3[j] < 0].std()

#Neurons of Layer 4
posm_feat4 = [0] * l4
negm_feat4 = [0] * l4
poss_feat4 = [0] * l4
negs_feat4 = [0] * l4

for j in range(l4):
    posm_feat4[j] = feat4[j][feat4[j] > 0].mean()
    negm_feat4[j] = feat4[j][feat4[j] < 0].mean()
    poss_feat4[j] = feat4[j][feat4[j] > 0].std()
    negs_feat4[j] = feat4[j][feat4[j] < 0].std()

In [ ]:
# Mean value Quantization

for x in range(l1):
    for y in range(784):
        if (feat1[x, y] > 0):
            feat1[x,y] = posm_feat1[x]
        if (feat1[x, y] < 0):
            feat1[x,y] = negm_feat1[x]
            
for x in range(l2):
    for y in range(l1):
        if (feat2[x, y] > 0):
            feat2[x,y] = posm_feat2[x]
        if (feat2[x, y] < 0):
            feat2[x,y] = negm_feat2[x]
            
for x in range(l3):
    for y in range(l2):
        if (feat3[x, y] > 0):
            feat3[x,y] = posm_feat3[x]
        if (feat3[x, y] < 0):
            feat3[x,y] = negm_feat3[x]
            
for x in range(l4):
    for y in range(l3):
        if (feat4[x, y] > 0):
            feat4[x,y] = posm_feat4[x]
        if (feat4[x, y] < 0):
            feat4[x,y] = negm_feat4[x]

In [ ]:
net1.save('examples/mlp/zeropr/quantization.caffemodel')

In [ ]:
for x in range(l1):
    print ((feat1[x] == posm_feat1[x]) | (feat1[x] == negm_feat1[x]) | (feat1[x] == 0)).sum()
for y in range(l2):
    print ((feat2[y] == posm_feat2[y]) | (feat2[y] == negm_feat2[y]) | (feat2[y] == 0)).sum()
for y in range(l3):
    print ((feat3[y] == posm_feat3[y]) | (feat3[y] == negm_feat3[y]) | (feat3[y] == 0)).sum()
for y in range(l4):
    print ((feat4[y] == posm_feat4[y]) | (feat4[y] == negm_feat4[y]) | (feat4[y] == 0)).sum()

In [ ]:
# same abolute values of posm and negm

for x in range(l1):
    for y in range(784):
        if (feat1[x, y] > 0):
            feat1[x,y] = (posm_feat1[x] - negm_feat1[x]) / 2
        if (feat1[x, y] < 0):
            feat1[x,y] = (negm_feat1[x] - posm_feat1[x]) / 2
            
for x in range(l2):
    for y in range(l1):
        if (feat2[x, y] > 0):
            feat2[x,y] = (posm_feat2[x] - negm_feat2[x]) / 2
        if (feat2[x, y] < 0):
            feat2[x,y] = (negm_feat2[x] - posm_feat2[x]) / 2
            
for x in range(l3):
    for y in range(l2):
        if (feat3[x, y] > 0):
            feat3[x,y] = (posm_feat3[x] - negm_feat3[x]) / 2
        if (feat3[x, y] < 0):
            feat3[x,y] = (negm_feat3[x] - posm_feat3[x]) / 2
            
for x in range(l4):
    for y in range(l3):
        if (feat4[x, y] > 0):
            feat4[x,y] = (posm_feat4[x] - negm_feat4[x]) / 2
        if (feat4[x, y] < 0):
            feat4[x,y] = (negm_feat4[x] - posm_feat4[x]) / 2

In [ ]:
net1.save('examples/mlp/zeropr/sameabs.caffemodel')

In [ ]:
for x in range(l1):
    print [(posm_feat1[x] + negm_feat1[x])/2]
           
for x in range(l2):
    print [(posm_feat2[x] + negm_feat2[x])/2]
    
for x in range(l3):
    print [(posm_feat3[x] + negm_feat3[x])/2]

In [ ]:
net1.params['ip1'][0].data